In [1]:
import re
import pandas as pd

In [2]:
### กำหนด data type ที่เหมาะสมกับ attribute values (Custom data types) ###


def guess_column_types(file_path, delimiter=',', has_headers=True):
    try:
        # Read the CSV file using the specified delimiter and header settings
        df = pd.read_csv(file_path, sep=delimiter,low_memory=False, header=0 if has_headers else None)

        # Initialize a dictionary to store column data types
        column_types = {}

        # Loop through columns and infer data types
        for column in df.columns:
            # sample_values = df[column].dropna().sample(min(5, len(df[column])), random_state=42)

            # Check for datetime format "YYYY-MM-DD HH:MM:SS"
            is_datetime = all(re.match(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', str(value)) for value in df[column])

            # Check for date format "YYYY-MM-DD"
            is_date = all(re.match(r'\d{4}-\d{2}-\d{2}', str(value)) for value in df[column])

            # Assign data type based on format detection
            if is_datetime:
                inferred_type = 'datetime64'
            elif is_date:
                inferred_type = 'date'
            else:
                inferred_type = pd.api.types.infer_dtype(df[column], skipna=True)

            column_types[column] = inferred_type

        return (True, column_types)  # Return success and column types
    except pd.errors.ParserError:
        return (False, str(e))  # Return error message

In [3]:
file_path = './LoanStats_web.csv'
result, column_types_or_error = guess_column_types(file_path)

if result:
    print("Column Types:", column_types_or_error)
else:
    print("Error:", column_types_or_error)


FileNotFoundError: [Errno 2] No such file or directory: './LoanStats_web.csv'

In [ ]:
# เปลี่ยน data type บางตัว ให้เหมาะสมกับ python's environment และ MSSQL
column_types_corrected = {col: ('datetime64' if t == 'date' else 'float64' if t == 'floating' else t) \
                          for col, t in column_types_or_error.items()}

In [ ]:
### นำ dictionary ที่บรรจุ ชื่อ col และ data type มาใช้เป็น parameter ของการทำ pd.read_csv ( ) อ่าน csv มาเป็น dataframe ###
raw_df = pd.read_csv(file_path, dtype=column_types_corrected)

In [ ]:
# คำนวน percentage ของ missing values ของแต่ละ col. ใน dataframe (raw_df)
missing_percentage = raw_df.isnull().mean() * 100

# กรอง columns ที่มี null เกินกว่า 30% ออกไป
columns_to_keep = missing_percentage[missing_percentage <= 30].index.tolist()
filteredCol_df = raw_df[columns_to_keep]

In [ ]:
raw_df = filteredCol_df

In [ ]:
raw_df.info()

In [ ]:
raw_df.columns

In [ ]:
acceptableMax_null = 26

In [ ]:
# สร้างรายการของคอลัมน์ที่มี Non-Null จำนวน 1,432,440
## selected_columns = [col for col in raw_df.columns if raw_df[col].notnull().sum() == 1432440]
selected_columns = [col for col in raw_df.columns if raw_df[col].isnull().sum() <= acceptableMax_null]

# แสดงคอลัมน์ที่เลือก
print("Selected columns:", selected_columns)

# สร้าง DataFrame ใหม่จากคอลัมน์ที่เลือก
df_selected = raw_df[selected_columns]


# ลบแถวที่มีค่า null ในคอลัมน์เหล่านั้น
noNull_df = df_selected.dropna()

# แสดงข้อมูลทั่วไปของ DataFrame หลังจากลบ null
noNull_df.info()

In [ ]:
# ทำสำเนาตัวแปร โดยหลังจากนี้ หากเกิดความเปลี่ยนแปลงกับ ตัวแปรใหม่ (df_prepared) จะไม่ส่งผลใดๆ ต่อตัวแปรเดิม (noNull_df)
df_prepared = noNull_df.copy()

# เปลี่ยน data type เป็น datetime สำหรับ col: issue_d
df_prepared['issue_d'] = pd.to_datetime(df_prepared['issue_d'], format='%b-%Y')

# นำเครื่องหมาย % ออกจากค่าใน col: int_rate แล้วเปลี่ยน data type เป็น float
if df_prepared['int_rate'].dtype == 'string':
    df_prepared['int_rate'] = df_prepared['int_rate'].str.rstrip('%').astype('float') / 100.0

In [ ]:
df_prepared.info()

In [ ]:
df_prepared.groupby('issue_d').size()

In [ ]:
df_prepared.groupby('int_rate').size()

In [ ]:
## dimension table เป็นสิ่งที่อยู่ใน data warehouse ไม่ใช่ python
## แต่เราจะใช้ python ไป create dim. table ใน data warehouse พร้อม insert ข้อมูลลงไปด้วย
## ด้วยเหตุนี้ ครูเอ้จึงเสนอ idea สำหรับการใช้ python พัฒนา dim. table ดังนี้
## (1) สร้าง dataframe ขึ้นมาใหม่ เพื่อทำหน้าที่เป็น dim. table โดยมีเฉพาะ col. ตาม dimensional model ที่ออกแบบไว้แล้ว
## (2) สร้าง col. ใหม่ขึ้นมาบน dataframe ข้อ (1) เพื่อจะใช้เป็น primary key สำหรับ dim. table

# ทำข้อ (1) สำหรับ home_ownership
home_ownership_dim = df_prepared[['home_ownership']].drop_duplicates().reset_index(drop=True)
home_ownership_dim.reset_index(inplace=True)
# ทำข้อ (2) สำหรับ home_ownership
home_ownership_dim['home_ownership_id'] = home_ownership_dim.index

# ทำข้อ (1) สำหรับ loan_status
loan_status_dim = df_prepared[['loan_status']].drop_duplicates().reset_index(drop=True)
loan_status_dim.reset_index(inplace=True)
# ทำข้อ (2) สำหรับ loan_status
loan_status_dim['loan_status_id'] = loan_status_dim.index

# ทำข้อ (1) สำหรับ สำหรับ issue_d
issue_d_dim = df_prepared[['issue_d']].drop_duplicates().reset_index(drop=True)
issue_d_dim['month'] = issue_d_dim['issue_d'].dt.month
issue_d_dim['year'] = issue_d_dim['issue_d'].dt.year
issue_d_dim.reset_index(inplace=True)
# ทำข้อ (2) สำหรับ สำหรับ issue_d
issue_d_dim['issue_d_id'] = issue_d_dim.index

In [ ]:
issue_d_dim

In [ ]:
loan_status_dim

In [ ]:
home_ownership_dim

In [ ]:
home_ownership_dim.set_index('home_ownership')

In [ ]:
home_ownership_dim.set_index('home_ownership')['home_ownership_id']

In [ ]:
home_ownership_dim.set_index('home_ownership')['home_ownership_id'].to_dict()

In [ ]:
loan_status_dim.set_index('loan_status')['loan_status_id'].to_dict()

In [ ]:
issue_d_dim.set_index('issue_d')['issue_d_id'].to_dict()

In [ ]:
## fact table เป็นสิ่งที่อยู่ใน data warehouse ไม่ใช่ python
## แต่เราจะใช้ python ไป create fact table ใน data warehouse พร้อม insert ข้อมูลลงไปด้วย
## ด้วยเหตุนี้ ครูเอ้จึงเสนอ idea สำหรับการใช้ python พัฒนา fact table ดังนี้
## (1) สร้าง python's dict. ขึ้นมาใช้ key mapping (สร้าง foreign key ของ fact table ตรงกับ primary key ของ dim. table)
## (2) dataframe ขึ้นมาใหม่ เพื่อทำหน้าที่เป็น fact table โดยนำ key mapping ตามข้อ 1 มาใช้ด้วย

In [ ]:
# ทำข้อ (1)
home_ownership_map = home_ownership_dim.set_index('home_ownership')['home_ownership_id'].to_dict()
loan_status_map = loan_status_dim.set_index('loan_status')['loan_status_id'].to_dict()
issue_d_map = issue_d_dim.set_index('issue_d')['issue_d_id'].to_dict()

In [ ]:
# ทำข้อ (2)
loans_fact = df_prepared.copy()
loans_fact['home_ownership_id'] = loans_fact['home_ownership'].map(home_ownership_map)
loans_fact['loan_status_id'] = loans_fact['loan_status'].map(loan_status_map)
loans_fact['issue_d_id'] = loans_fact['issue_d'].map(issue_d_map)

# เลือกคอลัมน์ที่จำเป็นสำหรับ Fact Table
loans_fact = loans_fact[['application_type','loan_amnt', 'funded_amnt', 'term', 'int_rate', 'installment'\
                         , 'home_ownership_id', 'loan_status_id', 'issue_d_id']]

In [ ]:
loans_fact.info()

In [ ]:
### เริ่มต้นเปลี่ยนเป็น raw cell ชั่วคราว ###

In [ ]:
## [optional] ทดสอบการ join ทุก dataframe ทั้ง fact และ dim. เข้าด้วยกัน

# Join `loans_fact` กับ `home_ownership_dim` โดยใช้ 'home_ownership_id'
loans_fact_with_home_ownership = pd.merge(loans_fact, home_ownership_dim, on='home_ownership_id', how='left', suffixes=('', '_home_ownership'))

# Join ผลลัพธ์กับ `loan_status_dim` โดยใช้ 'loan_status_id'
loans_fact_with_home_ownership_with_loan_status = pd.merge(loans_fact_with_home_ownership, loan_status_dim, on='loan_status_id', how='left', suffixes=('', '_loan_status'))

# Join ผลลัพธ์กับ `issue_d_dim` โดยใช้ 'issue_d_id'
final_df = pd.merge(loans_fact_with_home_ownership_with_loan_status, issue_d_dim, on='issue_d_id', how='left', suffixes=('', '_issue_d'))


In [ ]:
## [optional] ผลลัพธ์จากการ join คือ final_df ต้องมี row เท่ากันกับ dataframe เดิม
print(f"จำนวนแถวใน noNull_df: {noNull_df.shape[0]}")
print(f"จำนวนแถวใน final_df: {final_df.shape[0]}")

In [ ]:
## [optional] ผลลัพธ์จากการ join คือ final_df ต้องมีค่าของ measure เท่ากันกับ dataframe เดิม
print("การเปรียบเทียบค่า loan_amnt จาก dataframe เดิม กับ fact:")
print(noNull_df['funded_amnt'].head())
print(final_df['funded_amnt'].head())


In [ ]:
## [optional] ผลลัพธ์จากการ join คือ final_df ต้องไม่มี NULL เลย
print("จำนวนค่า Null ใน final_df หลังจากการ join:")
print(final_df.isnull().sum())


In [ ]:
### สิ้นสุดเปลี่ยนเป็น raw cell ชั่วคราว ###

In [ ]:
server = '34.136.225.236'
database = 'loanDW'
username = 'SA'
password = 'Passw0rd123456'

In [ ]:
issue_d_dim.drop(columns=['index'], inplace=True)

In [ ]:
loan_status_dim.drop(columns=['index'], inplace=True)

In [ ]:
home_ownership_dim.drop(columns=['index'], inplace=True)